In [1]:


import clingo
import clingo.ast

In [2]:
program = """

domA(0).

domA(A) :- domA(A).

"""

In [3]:
unground_reified = """

rule(head(literal(sign(0), atom(symbol(function(name("domA"), arguments(term(symbol(0))), external(0)))))), body(elements())).
rule(head(literal(sign(0), atom(symbol(function(name("domA"), arguments(variable(name("A"))), external(0)))))), body(elements(literal(sign(0), atom(symbol(function(name("domA"), arguments(variable(name("A"))), external(0)))))))).


"""

In [4]:
class NonGroundReifyTransformer(clingo.ast.Transformer):
    pos = clingo.ast.Position('<string>', 1, 1)
    loc = clingo.ast.Location(pos, pos)

    def _ast_sym(self, sym: clingo.Symbol):
        return clingo.ast.SymbolicTerm(self.loc, sym)

    def _ast_str(self, string: str):
        return self._ast_sym(clingo.String(string))

    def _ast_num(self, num: int):
        return self._ast_sym(clingo.Number(num))

    def visit_SymbolicTerm(self, term: clingo.ast.AST):
        print("Term:", term)
        symbol = term.symbol
        symbol_node = clingo.ast.Function(self.loc, 'symbol',
                                          [self._ast_sym(symbol)],
                                          False)
        print("Meta-Term:", symbol_node)
        return symbol_node

    def visit_Variable(self, variable: clingo.ast.AST):
        print("Variable:", variable)
        name = variable.name
        variable_node = clingo.ast.Function(self.loc, 'variable', [
            clingo.ast.Function(self.loc, 'name', [
                self._ast_str(name)
            ], False)], False)
        print("Meta-Variable:", variable_node)
        return variable_node

    def visit_Function(self, function: clingo.ast.AST):
        print("Function:", function)
        name = function.name
        external = function.external
        meta_function = self.visit_children(function)
        arguments = meta_function.get('arguments', ())
        arguments_sub = None
        if arguments:
            arguments_sub = clingo.ast.Function(self.loc, 'arguments', arguments, False)
        else:
            arguments_sub = self._ast_sym(clingo.Function('arguments'))
        function_node = clingo.ast.Function(self.loc, 'function', [
            clingo.ast.Function(self.loc, 'name', [self._ast_str(name)], False),
            arguments_sub,
            clingo.ast.Function(self.loc, 'external', [self._ast_num(external)], False)
        ], False)
        print("Meta-Function:", function_node)
        return function_node

    def visit_Literal(self, literal: clingo.ast.AST):
        print("Literal:", literal)
        sign = literal.sign
        meta_literal = self.visit_children(literal)
        atom = meta_literal['atom'].symbol

        literal_node = clingo.ast.Function(self.loc, 'literal', [
            clingo.ast.Function(self.loc, 'sign', [self._ast_num(sign)], False),
            clingo.ast.Function(self.loc, 'atom', [
                clingo.ast.Function(self.loc, 'symbol', [
                    atom
                ], False),
            ], False)
        ], False)
        print("Meta-Literal:", literal_node)
        return literal_node

    def visit_Rule(self, rule: clingo.ast.AST):
        print("Rule:", rule)
        meta_rule = self.visit_children(rule)
        head = clingo.ast.Function(self.loc, 'head', [meta_rule['head']], False)
        body = clingo.ast.Function(self.loc, 'body', [
            clingo.ast.Function(self.loc, 'elements', meta_rule.get('body', ()), False)], False)
        rule_head = clingo.ast.Literal(self.loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(
            clingo.ast.Function(self.loc, 'rule', (head, body), False)))
        rule_node = clingo.ast.Rule(self.loc, rule_head, ())
        print("Meta-Rule:", rule_node)
        return rule_node


In [5]:
ngrt = NonGroundReifyTransformer()

clingo.ast.parse_string("dom(0). dom(1) :- dom(A).", lambda stm: print("-" * 80, ngrt.visit(stm), "-" * 80, sep='\n'))

--------------------------------------------------------------------------------
#program base.
--------------------------------------------------------------------------------
Rule: dom(0).
Literal: dom(0)
Function: dom(0)
Term: 0
Meta-Term: symbol(0)
Meta-Function: function(name("dom"),arguments(symbol(0)),external(0))
Meta-Literal: literal(sign(0),atom(symbol(function(name("dom"),arguments(symbol(0)),external(0)))))
Meta-Rule: rule(head(literal(sign(0),atom(symbol(function(name("dom"),arguments(symbol(0)),external(0)))))),body(elements)).
--------------------------------------------------------------------------------
rule(head(literal(sign(0),atom(symbol(function(name("dom"),arguments(symbol(0)),external(0)))))),body(elements)).
--------------------------------------------------------------------------------
Rule: dom(1) :- dom(A).
Literal: dom(1)
Function: dom(1)
Term: 1
Meta-Term: symbol(1)
Meta-Function: function(name("dom"),arguments(symbol(1)),external(0))
Meta-Literal: litera

In [6]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0


In [7]:
ctl.add('base', [], unground_reified)

In [8]:
ctl.ground([('base', [])])

In [9]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(atoms=True)
        models.append(symbols)

In [10]:
print(' '.join(map(str, models[0])))

rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements)) rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0)))))),body(elements(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0))))))))
